In [2]:
import geopandas as gpd



#Directory CHM france 2018, 2019, 2020
chm_2020 = '/Users/arthurcalvi/Data/Disturbances_maps/FORMS/2020_l93.tif'

#Directory folder with the tiles
tiles = '/Users/arthurcalvi/Data/species/validation/tiles'

# Load the data
result_gdf = gpd.read_parquet("/Users/arthurcalvi/Data/species/validation/val_train_tiles.parquet")
print(result_gdf.crs)



{"$schema": "https://proj.org/schemas/v0.5/projjson.schema.json", "type": "ProjectedCRS", "name": "WGS 84 / UTM zone 31N", "base_crs": {"name": "WGS 84", "datum_ensemble": {"name": "World Geodetic System 1984 ensemble", "members": [{"name": "World Geodetic System 1984 (Transit)"}, {"name": "World Geodetic System 1984 (G730)"}, {"name": "World Geodetic System 1984 (G873)"}, {"name": "World Geodetic System 1984 (G1150)"}, {"name": "World Geodetic System 1984 (G1674)"}, {"name": "World Geodetic System 1984 (G1762)"}, {"name": "World Geodetic System 1984 (G2139)"}], "ellipsoid": {"name": "WGS 84", "semi_major_axis": 6378137, "inverse_flattening": 298.257223563}, "accuracy": "2.0", "id": {"authority": "EPSG", "code": 6326}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "id": {"authority": "EP

,perc,set,NomSER,geometry,color,year,perc_year
0,0.025600,training,Ouest-Bretagne et Nord-Cotentin,"POLYGON ((135901.068 5271036.264, 135901.068 5...",blue,2020.0,0.250000
1,0.023959,training,Ouest-Bretagne et Nord-Cotentin,"POLYGON ((160901.068 5346036.264, 160901.068 5...",blue,2022.0,0.996756
2,0.020800,training,Ouest-Bretagne et Nord-Cotentin,"POLYGON ((245901.068 5381036.264, 245901.068 5...",blue,2022.0,0.999344
3,0.019061,training,Ouest-Bretagne et Nord-Cotentin,"POLYGON ((-59098.932 5386036.264, -59098.932 5...",blue,NaN,0.000000
4,0.028800,validation,Ouest-Bretagne et Nord-Cotentin,"POLYGON ((245901.068 5436036.264, 245901.068 5...",yellow,2022.0,0.999463


In [19]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling, transform_geom
import os
import numpy as np
from shapely.geometry import box, mapping
from tqdm import tqdm

def crop_and_save_chm(chm_path: str, tiles_path: str, gdf: gpd.GeoDataFrame):
    """
    Crop the CHM for each tile using the geometry of the corresponding RGB file and save it in the corresponding folder.

    Args:
        chm_path (str): Path to the CHM .tif file.
        tiles_path (str): Path to the directory containing the tile folders.
        gdf (gpd.GeoDataFrame): GeoDataFrame containing the AOIs.
    """
    with rasterio.open(chm_path) as src:
        chm_crs = src.crs
        # Reproject gdf to match the CRS of the CHM
        gdf = gdf.to_crs(chm_crs)

        for idx, row in tqdm(gdf.iterrows()):
            # Match the correct folder
            tile_folders = sorted([f for f in os.listdir(tiles_path) if f.startswith(f'tile_') and f.split('_')[1].isdigit()], key=lambda x: int(x.split('_')[1]))
            matching_tile_folder = next((f for f in tile_folders if int(f.split('_')[1]) == idx), None)
            if matching_tile_folder:
                tile_folder = os.path.join(tiles_path, matching_tile_folder, 'tree_map')
                os.makedirs(tile_folder, exist_ok=True)
                output_path = os.path.join(tile_folder, 'CHM2020.tif')

                print(f"writing {output_path} for index {idx}")
                
                # Use the geometry of the RGB file for cropping
                rgb_folder = os.path.join(tiles_path, matching_tile_folder, 'rgb')
                for file in os.listdir(rgb_folder):
                    if file.endswith(".tif"):
                        with rasterio.open(os.path.join(rgb_folder, file)) as rgb_src:
                            target_crs = rgb_src.crs
                            target_bounds = rgb_src.bounds
                            target_geometry = box(target_bounds.left, target_bounds.bottom, target_bounds.right, target_bounds.top)
                            target_geometry = mapping(target_geometry)
                            target_transform = rgb_src.transform
                            target_width = rgb_src.width
                            target_height = rgb_src.height
                            # Transform the geometry to the CRS of the CHM
                            if target_crs != chm_crs:
                                target_geometry = transform_geom(target_crs, chm_crs, target_geometry)
                            break
                
                # Crop using the target geometry
                out_image, out_transform = mask(src, [target_geometry], crop=True)
                out_meta = src.meta.copy()
                out_meta.update({
                    "driver": "GTiff",
                    "height": out_image.shape[1],
                    "width": out_image.shape[2],
                    "transform": out_transform,
                    "crs": chm_crs,
                    "count": 1,
                    "nodata": src.nodata
                })
                
                if chm_crs != target_crs:

                    out_meta.update({
                        "crs": target_crs,
                        "transform": target_transform,
                        "width": target_width,
                        "height": target_height,
                        "nodata": src.nodata
                    })
                    with rasterio.open(output_path, "w", **out_meta) as dst:
                        reproject(
                            source=out_image,
                            destination=rasterio.band(dst, 1),
                            src_transform=out_transform,
                            src_crs=chm_crs,
                            dst_transform=target_transform,
                            dst_crs=target_crs,
                            resampling=Resampling.bilinear
                        )
                else:
                    with rasterio.open(output_path, "w", **out_meta) as dst:
                        dst.write(out_image[0], 1)


# Directory CHM France 2020
chm_2020 = '/Users/arthurcalvi/Data/Disturbances_maps/FORMS/2020_l93.tif'

# Directory folder with the tiles
tiles = '/Users/arthurcalvi/Data/species/validation/tiles'

# Load the data
result_gdf = gpd.read_parquet("/Users/arthurcalvi/Data/species/validation/val_train_tiles.parquet")
print(result_gdf.crs)

crop_and_save_chm(chm_2020, tiles, result_gdf)


{"$schema": "https://proj.org/schemas/v0.5/projjson.schema.json", "type": "ProjectedCRS", "name": "WGS 84 / UTM zone 31N", "base_crs": {"name": "WGS 84", "datum_ensemble": {"name": "World Geodetic System 1984 ensemble", "members": [{"name": "World Geodetic System 1984 (Transit)"}, {"name": "World Geodetic System 1984 (G730)"}, {"name": "World Geodetic System 1984 (G873)"}, {"name": "World Geodetic System 1984 (G1150)"}, {"name": "World Geodetic System 1984 (G1674)"}, {"name": "World Geodetic System 1984 (G1762)"}, {"name": "World Geodetic System 1984 (G2139)"}], "ellipsoid": {"name": "WGS 84", "semi_major_axis": 6378137, "inverse_flattening": 298.257223563}, "accuracy": "2.0", "id": {"authority": "EPSG", "code": 6326}}, "coordinate_system": {"subtype": "ellipsoidal", "axis": [{"name": "Geodetic latitude", "abbreviation": "Lat", "direction": "north", "unit": "degree"}, {"name": "Geodetic longitude", "abbreviation": "Lon", "direction": "east", "unit": "degree"}]}, "id": {"authority": "EP

1it [00:00,  8.51it/s]

writing /Users/arthurcalvi/Data/species/validation/tiles/tile_0_20180101_20221231_Ouest-Bretagne_et_Nord-Cotentin_training/tree_map/CHM2020.tif for index 0
writing /Users/arthurcalvi/Data/species/validation/tiles/tile_1_20200102_20241230_Ouest-Bretagne_et_Nord-Cotentin_training/tree_map/CHM2020.tif for index 1


3it [00:00,  8.86it/s]

writing /Users/arthurcalvi/Data/species/validation/tiles/tile_2_20200102_20241230_Ouest-Bretagne_et_Nord-Cotentin_training/tree_map/CHM2020.tif for index 2
writing /Users/arthurcalvi/Data/species/validation/tiles/tile_4_20200102_20241230_Ouest-Bretagne_et_Nord-Cotentin_validation/tree_map/CHM2020.tif for index 4


...

writing /Users/arthurcalvi/Data/species/validation/tiles/tile_80_20200102_20241230_Corse_occidentale_training/tree_map/CHM2020.tif for index 80
writing /Users/arthurcalvi/Data/species/validation/tiles/tile_81_20200102_20241230_Corse_occidentale_training/tree_map/CHM2020.tif for index 81
writing /Users/arthurcalvi/Data/species/validation/tiles/tile_82_20200102_20241230_Corse_occidentale_training/tree_map/CHM2020.tif for index 82


85it [00:05, 15.78it/s]

writing /Users/arthurcalvi/Data/species/validation/tiles/tile_83_20200102_20241230_Corse_occidentale_training/tree_map/CHM2020.tif for index 83
writing /Users/arthurcalvi/Data/species/validation/tiles/tile_84_20200102_20241230_Corse_occidentale_validation/tree_map/CHM2020.tif for index 84
writing /Users/arthurcalvi/Data/species/validation/tiles/tile_85_20200102_20241230_Corse_occidentale_validation/tree_map/CHM2020.tif for index 85


88it [00:05, 15.99it/s]

writing /Users/arthurcalvi/Data/species/validation/tiles/tile_86_20200102_20241230_Corse_occidentale_validation/tree_map/CHM2020.tif for index 86
writing /Users/arthurcalvi/Data/species/validation/tiles/tile_87_20200102_20241230_Corse_occidentale_validation/tree_map/CHM2020.tif for index 87
